In [1]:
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode
from pyspark.sql.types import StringType, IntegerType, BooleanType, FloatType, TimestampType

DATA_DIR = Path("data/ml-latest-small")

In [2]:
spark = SparkSession.builder\
    .master("local")\
    .appName("Word Count")\
    .getOrCreate()

In [3]:
# TODO: Spark schemas?
def read_df(file: str, types: dict):
    df = spark.read\
        .format("csv")\
        .option("header", "true")\
        .load(str(DATA_DIR / file))
    return set_dtypes(df, types)

def set_dtypes(df, types: dict):
    for column, dtype in types.items():
        dtype = {
            str: StringType(),
            int: IntegerType(),
            float: FloatType(),
            bool: BooleanType(),
            "time": TimestampType() 
        }[dtype]
        df = df.withColumn(column, col(column).cast(dtype))
    return df

df_links = read_df("links.csv", {
    "movieId": int,
    "imdbId": int,
    "tmdbId": int
})
df_movies = read_df("movies.csv", {
    "movieId": int,
    "title": str,
    "genres": str
})
df_ratings = read_df("ratings.csv", {
    "userId": int,
    "movieId": int,
    "rating": float,
    "timestamp": "time"
})
df_tags = read_df("tags.csv", {
    "userId": int,
    "movieId": int,
    "tag": str,
    "timestamp": "time"
})

In [4]:
# Given a user, get the number of movies watched per genre
def search_user(user_id: int):
    rated_movies = df_ratings.filter(df_ratings.userId == user_id).select("movieId").distinct()
    tagged_movies = df_tags.filter(df_tags.userId == user_id).select("movieId").distinct()
    movies = rated_movies.union(tagged_movies).distinct().join(df_movies, on=["movieId"], how="inner")
    movies = movies.select(movies.movieId, explode(split(movies.genres, "\|")).alias("genre"))
    movies = movies.groupBy("genre").count()
    return movies.collect()
search_user(1)

[Row(genre='Crime', count=45),
 Row(genre='Romance', count=26),
 Row(genre='Thriller', count=55),
 Row(genre='Adventure', count=85),
 Row(genre='Drama', count=68),
 Row(genre='War', count=22),
 Row(genre='Fantasy', count=47),
 Row(genre='Mystery', count=18),
 Row(genre='Musical', count=22),
 Row(genre='Animation', count=29),
 Row(genre='Film-Noir', count=1),
 Row(genre='Horror', count=17),
 Row(genre='Western', count=7),
 Row(genre='Comedy', count=83),
 Row(genre='Children', count=42),
 Row(genre='Action', count=90),
 Row(genre='Sci-Fi', count=40)]

In [5]:
df_ratings.filter("userId = 21").filter("movieId=1").collect()

[Row(userId=21, movieId=1, rating=3.5, timestamp=None)]

In [6]:
# Given a list of users, search all movies watched by each user
def search_movies_by_users(user_ids: [int]):
    rated_movies = df_ratings.filter(df_ratings.userId.isin(user_ids))
    tagged_movies = df_tags.filter(df_tags.userId.isin(user_ids))
    movies = rated_movies.join(tagged_movies, on=["userId", "movieId"], how="outer")
    movies = movies.select("userId", "movieId").distinct()
    movies = movies.groupBy("movieId").count().join(df_movies, on=["movieId"]).select("title", "count")
    return movies.collect()
search_movies_by_users([1, 21])

Row(title='Pride & Prejudice (2005)', count=1),
 Row(title='Who Framed Roger Rabbit? (1988)', count=1),
 Row(title='Pink Floyd: The Wall (1982)', count=1),
 Row(title='Logan (2017)', count=1),
 Row(title='Teenage Mutant Ninja Turtles III (1993)', count=1),
 Row(title='Ratatouille (2007)', count=1),
 Row(title='Transformers: The Movie (1986)', count=1),
 Row(title='Kingsman: The Secret Service (2015)', count=1),
 Row(title='The Interview (2014)', count=1),
 Row(title='Moonraker (1979)', count=1),
 Row(title='Monty Python and the Holy Grail (1975)', count=1),
 Row(title='Jurassic Park III (2001)', count=1),
 Row(title='Predator (1987)', count=1),
 Row(title='Dennis the Menace (1993)', count=1),
 Row(title='Collateral (2004)', count=1),
 Row(title='Welcome to Woop-Woop (1997)', count=1),
 Row(title='Big Lebowski, The (1998)', count=1),
 Row(title='Frozen (2013)', count=1),
 Row(title='Die Hard: With a Vengeance (1995)', count=1),
 Row(title='Terminator, The (1984)', count=2),
 Row(title='

In [7]:
# Search movie by id/title, show the average rating, the number of users that have watched the movie
# TODO: Need to translate between ID & Title
def search_movies_watched_by_id(movie_id: int):
    rated_movies = df_ratings.where(f"movieId = {movie_id}")
    tagged_movies = df_tags.where(f"movieId = {movie_id}")
    movies = rated_movies.join(tagged_movies, on=["userId"], how="outer")
    movies = movies.select("userId").distinct()
    return movies.count()
search_movies_watched_by_id(1) # Toy Story

215

In [8]:
# Search movie by id/title, show the average rating, the number of users that have watched the movie
def search_movies_avg_rating_by_id(movie_id: int):
    rated_movies = df_ratings.where(f"movieId = {movie_id}").agg({"rating": "avg"})
    return rated_movies.first()[0]
search_movies_avg_rating_by_id(1) # Toy Story

3.9209302325581397

In [25]:
# Given a list of genres, search all movies belonging each genre
def search_movies_by_genre(genres: [str]):
    movies = df_movies.select("movieId", "title", explode(split(df_movies.genres, "\|")).alias("genre"))
    movies = movies.filter(movies.genre.isin(genres)).dropDuplicates(subset=["movieId"])
    return [row.title for row in movies.select("title").collect()]
search_movies_by_genre(["Action", "Crime"])

rrah for Chivalry (Hao xia) (1979)',
 'Saw V (2008)',
 'Colombiana (2011)',
 'High Rise (2015)',
 'Jumanji: Welcome to the Jungle (2017)',
 'Mulholland Falls (1996)',
 'Army of Darkness (1993)',
 'Lodger: A Story of the London Fog, The (1927)',
 'Killing, The (1956)',
 'Sherlock Holmes and the Secret Weapon (1942)',
 'Appleseed (Appurushîdo) (2004)',
 'X-Men: The Last Stand (2006)',
 'Gamers, The: Dorkness Rising (2008)',
 'Ghost in the Shell (2017)',
 'Nothing to Lose (1994)',
 'Femme Nikita, La (Nikita) (1990)',
 'Newton Boys, The (1998)',
 'Thing, The (1982)',
 'Conquest of the Planet of the Apes (1972)',
 'Guns of Navarone, The (1961)',
 'Kiss of the Dragon (2001)',
 'Grind (2003)',
 'High Sierra (1941)',
 'Gomorrah (Gomorra) (2008)',
 'Green Zone (2010)',
 'Middle Men (2009)',
 'Dragon Ball: The Curse Of The Blood Rubies (Doragon bôru: Shenron no densetsu) (1986)',
 'LEGO Batman: The Movie - DC Heroes Unite (2013)',
 'Dragonheart 2: A New Beginning (2000)',
 'Copycat (1995)',
 'Ta